## Calculation the mean and standard deviation of the generated spectra for normalizing  the spectra passed to the model

In [1]:
import math
import tensorflow as tf

Ranges of parameters in units and constants used in experiment.

In [2]:
sp_min, sp_max = -10., 10.
fr_min, fr_max = 0., 256.
dc_min, dc_max = 0.1, 10.
am_min, am_max = 0.1, 5.

series = 20
dwmin, dwmax = -10., 10.

Number of pixels along the speed and frequency dimension.

In [3]:
sp_pixels, fr_pixels = 256, 256

Auxiliary arrays for generating spectrum in tensorflow.

In [4]:
t = tf.complex(tf.range(2 * fr_pixels, dtype = tf.float32) / fr_pixels / 2., 0.)
a = tf.range(series, dtype = tf.float32)
b = 2. * math.pi * \
    tf.complex(0., tf.range(dwmin, dwmax, (dwmax - dwmin) / sp_pixels, dtype = tf.float32))[:, None] * t

Function to take peak parameters and returns the resulting spectrum.

In [5]:
def get(sp_values, fr_values, dc_values, am_values):
    f = a[:, None] * sp_values + fr_values + fr_pixels / 2.
    fid = tf.complex(am_values, 0.) * tf.exp(2. * math.pi * t[:, None, None] * (tf.complex(-dc_values, f))) * \
          tf.cast(tf.logical_and(0.8 * fr_pixels / 2. <= f, f <= 2. * fr_pixels - 0.8 * fr_pixels / 2.), 
                  tf.complex64)
    fid = tf.reduce_sum(fid, 2)
    fid = tf.concat([fid[: 1] / 2., fid[1:]], 0)
    p = tf.exp(-b[:, :, None] * tf.complex(a, 0.)) * fid
    return tf.math.real(tf.signal.fft(tf.reduce_sum(p, 2)))[:, fr_pixels // 2 : 2 * fr_pixels - fr_pixels // 2]

Function to draw random peak parameters and yield the resulting spectrum.

In [6]:
def generate():
    while True:
        #Random number of peaks from 1 to 9
        count = tf.random.uniform([], 1, 10, dtype = tf.int32)
        #Random peak parameters in units used in experiment
        sp_values = tf.random.uniform([count], sp_min, sp_max)
        fr_values = tf.random.uniform([count], fr_min, fr_max)
        dc_values = tf.random.uniform([count], dc_min, dc_max)
        am_values = tf.random.uniform([count], am_min, am_max)
        #Spectrum resulting from these parameters
        spectrum = get(sp_values, fr_values, dc_values, am_values)
        yield spectrum

Batched dataset of spectra from function of generate.

In [7]:
dataset = tf.data.Dataset.from_generator(generate,
                                         output_signature = tf.TensorSpec([sp_pixels, fr_pixels]))
dataset = dataset.batch(4096)

### Calculation the mean and standard deviation of the generated spectra

In [8]:
spectrum = next(iter(dataset))
print("MEANspectrum: %4f" %(tf.math.reduce_mean(spectrum).numpy()))
print("STDspectrum: %4f" %(tf.math.reduce_std(spectrum).numpy()))

MEANspectrum: 179.778900
STDspectrum: 215.897995
